In [1]:
# Import libraries
from functions import *

import json
import os
import subprocess
import time

# Creating a P2WSH multisig transaction

In this section we'll create a P2WSH transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and we'll test it using bitcoin core in regtest mode.

## Reading
- Andreas Antonopoulos - Mastering Bitcoin Chapter 7
- Jimmy Song - Programming Bitcoin Chapters 13

## Setup 

### Requirements
For this exercise we'll need Bitcoin Core (v22 or higher) with the application data is stored in 
```$HOME/Library/Application Support/Bitcoin```.

### Setup bitcoind in regtest
Start up regtest mode, delete any regtest network history so we are starting from scratch. Create a wallet so that we can fund our first output using the `sendtoaddress` command. Mine 101 blocks so that the mining reward first block will have sufficient block maturity (100 blocks) to spend from.

In [2]:
# start up bitcoind in regtest and mine 101 blocks

# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(2) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# Start up bitcoind in regtest mode
os.system("bitcoind -regtest -daemon -fallbackfee=0.0002")
time.sleep(1.5)

# Create a new wallet and address that we can mine blocks so that we can fund our transactions
wallet = subprocess.getoutput("bitcoin-cli -regtest createwallet mywallet")
address = subprocess.getoutput("bitcoin-cli -regtest getnewaddress")

# Generate 101 blocks so that the first block's block reward reaches maturity
result = subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 101 {addr}".format(addr=address))

# Check that we were able to mine 101 blocks
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")
assert(blockcount == "101")

### Create a P2WSH multisig UTXO

In order to create a transaction spending from a P2WSH multisig UTXO, we'll first need to create the UTXO that is locked with a multisig script. For this example we'll use a 2 of 2 multisig script. Note that this type of output will be identical to funding transactions used for layer 2 lightning channels. 

We'll create a two pubkeys and manually conscruct the multisig script. Then we'll fund it using the bitcoind wallet created in the setup step.

#### Create a multisig redeemScript 
For more on this step, review the 'Bitcoin Script' notebook.

In [3]:
privkey1 = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
pubkey1 = privkey_to_pubkey(privkey1)

privkey2 = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
pubkey2 = privkey_to_pubkey(privkey2)

# "21" is the length of a 33 byte (compressed) pubkey in hex notation
redeemScript = bytes.fromhex(
    "52" 
    + "21"
    + pubkey1.hex()
    + "21"
    + pubkey2.hex()
    + "52"
    + "ae")

print(redeemScript.hex())

5221034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa2102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f2752ae


#### Convert the redeemScript to a P2WSH address

For more on this step, review the 'Addresses' notebook.

In [4]:
address_to_spend = script_to_p2wsh(redeemScript, "regtest")
print(address_to_spend)

bcrt1qpqn5k3h89nfv6cnrkvk3rt3g0zfhqfz23cxkgapsenj29ety5ckqyrn25s


#### Fund the 'sender' with 2.001 btc (0.001 btc is for the next tx fee)

In [5]:
txid_to_spend = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + address_to_spend + " 2.001")
print(txid_to_spend)

f46bbfbd025c36cc5fae08bb3b318417976dd91aa3635d1aa6814c0236ef0826


#### Find which output index the btc was sent to
Since we only sent 2.001 btc of the coinbase transaction (50 btc) to our address, bitcoind creates a change output to send the rest of the btc. By looking at the outputs we can see which is the change output and which was sent to our address. To do this in python we can do the following:

In [6]:
# Decode transaction to find which index to spend from

raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_to_spend)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
# print(decoded)

d = json.loads(decoded)
if d["vout"][0]["scriptPubKey"]["address"] == address_to_spend:
    index_to_spend = 0
elif d["vout"][1]["scriptPubKey"]["address"] == address_to_spend:
    index_to_spend = 1
else:
    raise Exception("couldn't find output")
print("index to spend from: " + str(index_to_spend))

index to spend from: 0


#### Mine a block so that the funding tx gets confirmed

In [7]:
subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 1 {addr}".format(addr=address));

## Spending a P2WSH (multisig) UTXO

Now that we have some funds locked up in a P2WSH multisig utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE`.

From our previous examples we know this corresponds to a scriptPubkey of `76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac`, so we will skip the decoding step. For more on decoding base58 addresses, refer to the 'Addresses' chapter.

In [8]:
receiver_spk = bytes.fromhex("76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac")

### Create an unsigned P2WSH transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [9]:
# Note we have already defined a few variables we need to create our transaction:
# The input utxo txid and index: `txid_to_spend` and `index_to_spend`
# The input private key and public key: `privkey1`, `pubkey1`, ...

# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("4444444444444444444444444444444444444444444444444444444444444444")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("76a914") + hash160(change_pubkey) + bytes.fromhex("88ac")

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the witness field is empty. This first step is necessary as the signature will cover the whole transaction (using SIGHASH_ALL). In a later chapter we will cover other sighash types and how they are signed.

In [10]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + pushbytes(output1_spk)
    + output2_value
    + pushbytes(output2_spk)
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  02000000012608ef36024c81a61a5d63a31ad96d971784313bbb08ae5fcc365c02bdbf6bf40000000000ffffffff0280d1f008000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914cc1b07838e387deacd0e5232e1e8b49f4c29e48488ac00000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (marker, flag, witness).

In [11]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + unsigned_tx.hex())
print(decoded)

{
  "txid": "ae2771e3413de6c9a54190288228e68cc925811235bee6d06a6b278eeb4dc8ad",
  "hash": "ae2771e3413de6c9a54190288228e68cc925811235bee6d06a6b278eeb4dc8ad",
  "version": 2,
  "size": 119,
  "vsize": 119,
  "weight": 476,
  "locktime": 0,
  "vin": [
    {
      "txid": "f46bbfbd025c36cc5fae08bb3b318417976dd91aa3635d1aa6814c0236ef0826",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 1.50000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 3bc28d6d92d9073fb5e3adf481795eaf446bceed OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE)#xlnzfr97",
        "hex": "76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac",
        "address": "mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE",
        "type": "pubkeyhash"
      }
    },
    {
      "value": 0.50000000,
      "n": 1,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 cc1b078

Segwit transactions have a new signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)

In [12]:
scriptcode = redeemScript

input_amount_sat = int(2.001 * 100_000_000)
value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(scriptcode)
    + scriptcode
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print(tx_digest_preimage.hex())

020000007367376725b71413899341be58b2832ebd606d41a55db982f19eeaecce9324913bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e706650442608ef36024c81a61a5d63a31ad96d971784313bbb08ae5fcc365c02bdbf6bf400000000475221034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa2102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f2752aea048ed0b00000000ffffffff7c27b92041484c01942ac85d0fe5ff31e92d1a54e1618e190ef3f316c4799ecf0000000001000000


Now we are ready to hash this transaction and produce two ecdsa signatures on it. 

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [13]:
# Create sigHash to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input privkey1
signing_key1 = ecdsa.SigningKey.from_string(privkey1, curve=ecdsa.SECP256k1) 
signature1 = signing_key1.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature1 = signature1 + bytes.fromhex("01")

# Sign the sigHash with the input privkey2
signing_key2 = ecdsa.SigningKey.from_string(privkey2, curve=ecdsa.SECP256k1) 
signature2 = signing_key2.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature2 = signature2 + bytes.fromhex("01")

# Combine the signatures into the final scriptSig
witness = (
    # indicate the number of stack items
    bytes.fromhex("04")
    + bytes.fromhex("00") # Add an extra "00" for the CheckMultiSig bug
    + pushbytes(signature1)
    + pushbytes(signature2)
    + pushbytes(redeemScript)
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  020000000001012608ef36024c81a61a5d63a31ad96d971784313bbb08ae5fcc365c02bdbf6bf40000000000ffffffff0280d1f008000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914cc1b07838e387deacd0e5232e1e8b49f4c29e48488ac0400483045022100b322c3a16ece76ae0d846a42cc8beab119ba7dd522f6563005ca200bccfb2e1c0220261a56a392e510d7a8bf9a7abd1dba7a1ece8a1073bea80cdda974557d6c50eb01473044022055281a875e70b96e694ecaffd8c446d90ffb4a59b1dd59000b00a8db84918eb902207229da03cfa139c2ecccaa85ce91edf5860ecce47ede5a806f6bb0c92ae0339d01475221034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa2102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f2752ae00000000


### Broadcase the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [14]:
# new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + signed_tx.hex())
new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest testmempoolaccept " + "'[\"" +  signed_tx.hex()+ "\"]'")

print(new_tx_txid)

[
  {
    "txid": "ae2771e3413de6c9a54190288228e68cc925811235bee6d06a6b278eeb4dc8ad",
    "wtxid": "617ba6449b79b64d6e797748a92c0e380634c3bcf4a4a3aef70e1b600cdbd804",
    "allowed": true,
    "vsize": 175,
    "fees": {
      "base": 0.00100000
    }
  }
]


In [15]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + signed_tx.hex())
print(decoded)

{
  "txid": "ae2771e3413de6c9a54190288228e68cc925811235bee6d06a6b278eeb4dc8ad",
  "hash": "617ba6449b79b64d6e797748a92c0e380634c3bcf4a4a3aef70e1b600cdbd804",
  "version": 2,
  "size": 340,
  "vsize": 175,
  "weight": 697,
  "locktime": 0,
  "vin": [
    {
      "txid": "f46bbfbd025c36cc5fae08bb3b318417976dd91aa3635d1aa6814c0236ef0826",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "3045022100b322c3a16ece76ae0d846a42cc8beab119ba7dd522f6563005ca200bccfb2e1c0220261a56a392e510d7a8bf9a7abd1dba7a1ece8a1073bea80cdda974557d6c50eb01",
        "3044022055281a875e70b96e694ecaffd8c446d90ffb4a59b1dd59000b00a8db84918eb902207229da03cfa139c2ecccaa85ce91edf5860ecce47ede5a806f6bb0c92ae0339d01",
        "5221034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa2102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f2752ae"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value"

## Quiz


## Answers


## Exercise

